<a href="https://colab.research.google.com/github/DaoLua/Opella./blob/Data_Logger/Data_Logger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tabula
import jpype
import pandas as pd
import re
import os
import sys
import subprocess
import glob
from PyPDF2 import PdfReader


# --- CHECK AND INSTALL DEPENDENCIES ---
required_packages = ['tabula-py', 'jpype1', 'PyPDF2', 'pandas', 're', 'os', 'subprocess', 'glob']
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Không tìm thấy thư viện {package}. Đang cài đặt...")
        %pip install {package}
        print(f"Đã cài đặt thành công thư viện {package}.")

print("\n\nChào mừng bạn đến với notebook phân tích dữ liệu từ các file PDF của thiết bị ghi nhiệt độ (datalogger).")
print("\nNotebook này sẽ giúp bạn:")
print("1.  Nhập thời gian nhập kho để lọc dữ liệu.")
print("2.  Xác định ngưỡng nhiệt độ cho phân tích 'Time Out of Refrigerator' (TOR).")
print("3.  Định dạng ngày tháng phù hợp với dữ liệu trong file PDF của bạn.")
print("4.  Trích xuất dữ liệu nhiệt độ từ các file PDF trong thư mục `/content/datalogger/`.")
print("5.  Thực hiện phân tích TOR để xác định các khoảng thời gian nhiệt độ vượt ngưỡng.")
print("6.  Tổng hợp và hiển thị kết quả chi tiết và tóm tắt.")
print("7.  Xuất kết quả ra tệp Excel.")
print("8.  Xóa các tệp PDF đã xử lý.")
print("\nVui lòng tương tác với các yêu cầu nhập liệu bên dưới để bắt đầu quá trình.\n")

# --- HELPER FUNCTIONS ---

def get_user_input():
    """Lấy thông tin đầu vào từ người dùng về thời gian cắt, ngưỡng nhiệt độ TOR và định dạng ngày tháng trong PDF."""
    print("\nBước 1: Thời gian nhập nguyên liệu vào kho")
    while True:
        try:
            input_date_str = input("Nhập ngày (DD/MM/YYYY): ")
            input_time_str = input("Nhập giờ (HH:MM): ")
            cutoff_str = f"{input_date_str} {input_time_str}:00"
            global_cutoff_datetime = pd.to_datetime(cutoff_str, format='%d/%m/%Y %H:%M:%S')
            break
        except ValueError:
            print("Sai định dạng. Vui lòng nhập lại.")

    print("\nBước 2: Giới hạn nhiệt độ của nguyên liệu")
    while True:
        try:
            tor_threshold_temp = float(input("Nhập nhiệt độ (°C): "))
            break
        except ValueError:
            print("Sai định dạng. Vui lòng nhập lại.")

    print("\nBước 3: Định dạng ngày tháng trong file PDF")
    print("Chọn một trong các định dạng sau:")
    print("1: yy/mm/dd")
    print("2: yy/dd/mm")
    print("3: dd/mm/yy")
    print("4: mm/dd/yy")

    while True:
        date_format_choice = input("Nhập lựa chọn của bạn (1-4): ")
        if date_format_choice == '1':
            pdf_date_format = '%y/%m/%d %H:%M:%S'
            pdf_date_label = 'yy/mm/dd'
            break
        elif date_format_choice == '2':
            pdf_date_format = '%y/%d/%m %H:%M:%S'
            pdf_date_label = 'yy/dd/mm'
            break
        elif date_format_choice == '3':
            pdf_date_format = '%d/%m/%y %H:%M:%S'
            pdf_date_label = 'dd/mm/yy'
            break
        elif date_format_choice == '4':
            pdf_date_format = '%m/%d/%y %H:%M:%S'
            pdf_date_label = 'mm/dd/yy'
            break
        else:
            print("Lựa chọn không hợp lệ. Vui lòng nhập số từ 1 đến 4.")

    return global_cutoff_datetime, tor_threshold_temp, pdf_date_format, pdf_date_label

def process_pdf(pdf_path, logger_id, global_cutoff_datetime, tor_threshold_temp, pdf_date_format, pdf_date_label):
    """Xử lý một tệp PDF để trích xuất dữ liệu và thực hiện phân tích TOR."""
    print(f"\nĐang xử lý file: {logger_id}")

    # --- COUNT TOTAL PAGES IN PDF ---
    try:
        with open(pdf_path, 'rb') as f:
            reader = PdfReader(f)
            total_pages = len(reader.pages)
        print(f"Tổng số trang trong {logger_id}: {total_pages}")
    except Exception as e:
        print(f"Lỗi đếm trang PDF {logger_id}: {e}")
        return None, None, None, None, None, None

    # --- EXTRACT TABLES FROM PDF ---
    try:
        dfs = tabula.read_pdf(pdf_path, pages=f"2-{total_pages}", stream=True, multiple_tables=True)
    except Exception as e:
        print(f"Lỗi đọc file PDF {logger_id}: {e}")
        return None, None, None, None, None, None

    # --- RESHAPE DATA FROM WIDE TO LONG FORMAT ---
    all_reshaped_chunks = []
    for page_df in dfs:
        for i in range(0, page_df.shape[1], 3):
            if i + 2 < page_df.shape[1]:
                chunk = page_df.iloc[:, i:i+3].copy()
                chunk.columns = ['raw_col1', 'raw_col2', 'raw_col3']
                all_reshaped_chunks.append(chunk)
    if not all_reshaped_chunks:
        # print(f"Không tìm thấy khối dữ liệu nào trong {logger_id}. Bỏ qua.")
        return None, None, None, None, None, None
    raw_long_df = pd.concat(all_reshaped_chunks, ignore_index=True)

    # --- CLEAN THE RESHAPED DATA ---
    all_records = []
    pattern = re.compile(r'(\d{2}/\d{2}/\d{2})?\s+(\d{2}:\d{2}:\d{2})\s+([\d\.\-]+)')

    combined_series = raw_long_df['raw_col1'].fillna('') + ' ' + \
                      raw_long_df['raw_col2'].fillna('') + ' ' + \
                      raw_long_df['raw_col3'].fillna('')

    for text in combined_series:
        matches = pattern.findall(text)
        for date_val, time_val, temp_val in matches:
            all_records.append({
                'date': date_val,
                'time': time_val,
                'temperature': temp_val})

    if not all_records:
        # print(f"Không tìm thấy bản ghi nào trong {logger_id}. Bỏ qua.")
        return None, None, None, None, None, None

    # --- CREATE THE DATAFRAME ---
    datalogger = pd.DataFrame(all_records)
    datalogger['datetime'] = pd.to_datetime(datalogger['date'] + ' ' + datalogger['time'], format=pdf_date_format, errors='coerce')
    datalogger['temperature'] = pd.to_numeric(datalogger['temperature'], errors='coerce')
    datalogger['temperature'] = datalogger['temperature'].astype('float16')
    datalogger.drop(columns=['date', 'time'], inplace=True)
    datalogger.dropna(subset=['datetime', 'temperature'], inplace=True)
    datalogger.sort_values('datetime', inplace=True, ignore_index=True)
    datalogger.drop_duplicates(subset=['datetime', 'temperature'], keep='first', inplace=True)
    datalogger = datalogger[['datetime', 'temperature']].reset_index(drop=True)

    if datalogger.empty:
        # print(f"Không còn dữ liệu hợp lệ nào trong {logger_id} sau khi làm sạch. Bỏ qua.")
        return None, None, None, None, None, None

    # --- FILTER THE DATAFRAME USING GLOBAL CUTOFF ---
    datalogger_filtered = datalogger[datalogger['datetime'] <= global_cutoff_datetime].copy()

    if datalogger_filtered.empty:
        # print(f"Không còn dữ liệu nào sau khi lọc cho {logger_id}. Bỏ qua phân tích TOR.")
        return None, None, None, None, None, None

    # --- Tính nhiệt độ Min và Max cùng thời gian của chúng cho dữ liệu đã lọc ---
    min_temp = datalogger_filtered['temperature'].min()
    max_temp = datalogger_filtered['temperature'].max()
    min_temp_datetime = datalogger_filtered[datalogger_filtered['temperature'] == min_temp]['datetime'].min()
    max_temp_datetime = datalogger_filtered[datalogger_filtered['temperature'] == max_temp]['datetime'].min()

    # --- TOR ANALYSIS ---
    print(f"Bước 5: Thực hiện phân tích TOR cho Logger ID {logger_id} với giới hạn nhiệt độ {tor_threshold_temp}°C...")
    datalogger_filtered['is_tor'] = datalogger_filtered['temperature'] > tor_threshold_temp

    tor_periods = []
    start_time = None
    datalogger_filtered = datalogger_filtered.reset_index(drop=True)

    for i in range(len(datalogger_filtered)):
        is_current_tor = datalogger_filtered.loc[i, 'is_tor']
        is_previous_tor = False if i == 0 else datalogger_filtered.loc[i-1, 'is_tor']
        is_next_tor = False if i == len(datalogger_filtered) - 1 else datalogger_filtered.loc[i+1, 'is_tor']

        if is_current_tor and not is_previous_tor:
            start_time = datalogger_filtered.loc[i, 'datetime']

        if is_current_tor and not is_next_tor and start_time is not None:
            end_time = datalogger_filtered.loc[i, 'datetime']
            tor_periods.append({'TOR start': start_time, 'TOR stop': end_time})
            start_time = None

    tor_df = pd.DataFrame(tor_periods)
    final_tor_df = None
    total_tor_file = pd.Timedelta(seconds=0)

    if not tor_df.empty:
        tor_df['Excursion duration'] = tor_df['TOR stop'] - tor_df['TOR start']
        tor_df['Excursion duration'] = tor_df['Excursion duration'].apply(lambda x: f'{x.days*24 + x.seconds//3600:02d}:{x.seconds%3600//60:02d}')

        tor_df = pd.merge(tor_df, datalogger_filtered[['datetime', 'temperature']], left_on='TOR start', right_on='datetime', how='left')
        tor_df.rename(columns={'temperature': 'temperature at tor start'}, inplace=True)
        tor_df.drop(columns=['datetime'], inplace=True)

        tor_df = pd.merge(tor_df, datalogger_filtered[['datetime', 'temperature']], left_on='TOR stop', right_on='datetime', how='left')
        tor_df.rename(columns={'temperature': 'temperature at tor stop'}, inplace=True)
        tor_df.drop(columns=['datetime'], inplace=True)

        final_tor_df = tor_df[['TOR start', 'temperature at tor start', 'TOR stop', 'temperature at tor stop', 'Excursion duration']]
        final_tor_df['Logger ID'] = logger_id.replace('.pdf', '')

        tor_df['Excursion duration_timedelta'] = tor_df['Excursion duration'].apply(lambda x: pd.to_timedelta(x + ':00'))
        total_tor_file = tor_df['Excursion duration_timedelta'].sum()

    return final_tor_df, total_tor_file, min_temp, min_temp_datetime, max_temp, max_temp_datetime

def format_timedelta_as_ddhhmm(td):
    """Định dạng đối tượng timedelta thành chuỗi 'dd:hh:mm'."""
    total_seconds = td.total_seconds()
    days = int(total_seconds // (24 * 3600))
    hours = int((total_seconds % (24 * 3600)) // 3600)
    minutes = int((total_seconds % 3600) // 60)
    return f"{days:02d}:{hours:02d}:{minutes:02d}"


# --- MAIN PROCESSING LOGIC ---

# --- SETUP DIRECTORY AND LIST PDF FILES ---
datalogger_dir = '/content/datalogger/'
os.makedirs(datalogger_dir, exist_ok=True)
pdf_files = [f for f in os.listdir(datalogger_dir) if f.endswith('.pdf')]

all_processed_dataframes = []
total_tor_data = []

# --- GET USER INPUT ---
global_cutoff_datetime, tor_threshold_temp, pdf_date_format, pdf_date_label = get_user_input()


# --- PROCESS EACH PDF FILE ---
for pdf_file in pdf_files:
    pdf_path = os.path.join(datalogger_dir, pdf_file)
    logger_id = pdf_file

    final_tor_df_single, total_tor_file_single, min_temp_single, min_temp_datetime_single, max_temp_single, max_temp_datetime_single = process_pdf(
        pdf_path, logger_id, global_cutoff_datetime, tor_threshold_temp, pdf_date_format, pdf_date_label
    )

    if final_tor_df_single is not None:
        all_processed_dataframes.append(final_tor_df_single)

    if min_temp_single is not None:
         total_tor_data.append({
            'Logger ID': logger_id.replace('.pdf', ''),
            'Total TOR': total_tor_file_single,
            'Min Temperature': min_temp_single,
            'Min Temp Datetime': min_temp_datetime_single,
            'Max Temperature': max_temp_single,
            'Max Temp Datetime': max_temp_datetime_single
        })


# --- COMBINE AND FINALIZE RESULTS ---
if all_processed_dataframes:
    combined_datalogger_df = pd.concat(all_processed_dataframes, ignore_index=True)
    combined_datalogger_df = combined_datalogger_df.reset_index(drop=True)

    total_tor_summary_df = pd.DataFrame(total_tor_data)

    # Định dạng và đặt nhãn Tổng TOR
    total_tor_summary_df['Tổng thời gian TOR (dd:hh:mm)'] = total_tor_summary_df['Total TOR'].apply(format_timedelta_as_ddhhmm)
    total_tor_summary_df.drop(columns=['Total TOR'], inplace=True)

    # Định dạng và đặt nhãn Ngày giờ nhiệt độ Min/Max
    total_tor_summary_df[f'Ngày giờ nhiệt độ Min ({pdf_date_label})'] = total_tor_summary_df['Min Temp Datetime'].dt.strftime(pdf_date_format.replace('%H:%M:%S', '%H:%M'))
    total_tor_summary_df.drop(columns=['Min Temp Datetime'], inplace=True)

    total_tor_summary_df[f'Ngày giờ nhiệt độ Max ({pdf_date_label})'] = total_tor_summary_df['Max Temp Datetime'].dt.strftime(pdf_date_format.replace('%H:%M:%S', '%H:%M'))
    total_tor_summary_df.drop(columns=['Max Temp Datetime'], inplace=True)

    # Thêm số thứ tự cho combined_datalogger_df
    combined_datalogger_df['Số thứ tự'] = combined_datalogger_df.groupby('Logger ID').cumcount() + 1

    # Định dạng và đặt nhãn Thời gian bắt đầu và Thời gian kết thúc TOR
    combined_datalogger_df[f'Thời gian bắt đầu TOR ({pdf_date_label})'] = combined_datalogger_df['TOR start'].dt.strftime(pdf_date_format.replace('%H:%M:%S', '%H:%M'))
    combined_datalogger_df.drop(columns=['TOR start'], inplace=True)

    combined_datalogger_df[f'Thời gian kết thúc TOR ({pdf_date_label})'] = combined_datalogger_df['TOR stop'].dt.strftime(pdf_date_format.replace('%H:%M:%S', '%H:%M'))
    combined_datalogger_df.drop(columns=['TOR stop'], inplace=True)

    # Thêm nhãn cho Thời gian vượt ngưỡng
    combined_datalogger_df['Thời gian vượt ngưỡng (hh:mm)'] = combined_datalogger_df['Excursion duration']
    combined_datalogger_df.drop(columns=['Excursion duration'], inplace=True)

    # Sắp xếp lại các cột cho combined_datalogger_df
    final_combined_df = combined_datalogger_df[['Logger ID', 'Số thứ tự', f'Thời gian bắt đầu TOR ({pdf_date_label})', 'temperature at tor start', f'Thời gian kết thúc TOR ({pdf_date_label})', 'temperature at tor stop', 'Thời gian vượt ngưỡng (hh:mm)']]

    print("--- Chi tiết các lần vượt ngưỡng TOR kết hợp với Logger ID, Số thứ tự và Nhiệt độ ---")
    display(final_combined_df)

    print("\n--- Tổng thời gian vượt ngưỡng (TOR) trên mỗi Logger ID, Nhiệt độ Min/Max và Ngày giờ ---")
    # Sắp xếp lại các cột cho hiển thị tóm tắt
    total_tor_summary_df = total_tor_summary_df[['Logger ID', 'Tổng thời gian TOR (dd:hh:mm)', 'Min Temperature', f'Ngày giờ nhiệt độ Min ({pdf_date_label})', 'Max Temperature', f'Ngày giờ nhiệt độ Max ({pdf_date_label})']]
    display(total_tor_summary_df)

    # --- EXPORT TO EXCEL ---
    output_excel_path = "/content/datalogger/tor_analysis_results.xlsx"
    with pd.ExcelWriter(output_excel_path) as writer:
        final_combined_df.to_excel(writer, sheet_name='Chi tiết vượt ngưỡng', index=False)
        total_tor_summary_df.to_excel(writer, sheet_name='Tóm tắt tổng TOR', index=False)

    print(f"\nKết quả đã được xuất thành công tại '{output_excel_path}'")

    # --- DELETE PDF FILES ---
    print("\nĐang xóa các tệp PDF từ /content/datalogger/")
    pdf_files_to_delete = glob.glob(os.path.join(datalogger_dir, '*.pdf'))
    for pdf_file_path in pdf_files_to_delete:
        try:
            os.remove(pdf_file_path)
            print(f"Đã xóa: {pdf_file_path}")
        except OSError as e:
            print(f"Lỗi khi xóa {pdf_file_path}: {e}")

else:
    print("Không có dữ liệu nào được xử lý từ các tệp PDF.")